# Word2Vec之Skip-Gram模型-中文文本版

下面代码将用TensorFlow实现Word2Vec中的Skip-Gram模型。

关于Skip-Gram模型请参考上一篇[知乎专栏文章](https://zhuanlan.zhihu.com/p/27234078)

# 1 导入包

In [1]:
import time
import numpy as np
import tensorflow as tf
import random
from collections import Counter

# 2 加载数据

数据集使用的是来自Matt Mahoney的维基百科文章，数据集已经被清洗过，去除了特殊符号等，并不是全量数据，只是部分数据，所以实际上最后训练出的结果很一般（语料不够）。

如果想获取更全的语料数据，可以访问以下网站，这是gensim中Word2Vec提供的语料：

- 来自Matt Mahoney预处理后的[文本子集](http://mattmahoney.net/dc/enwik9.zip)，里面包含了10亿个字符。
- 与第一条一样的经过预处理的[文本数据](http://dumps.wikimedia.org/enwiki/latest/enwiki-latest-pages-articles.xml.bz2)，但是包含了30个亿的字符。
- 多种语言的[训练文本](http://www.statmt.org/wmt11/translation-task.html#download)。
- [UMBC webbase corpus](http://ebiquity.umbc.edu/redirect/to/resource/id/351/UMBC-webbase-corpus)

In [2]:
with open('data/Javasplittedwords') as f:
    text = f.read()
words = text.split(' ')

# 3 数据预处理

数据预处理过程主要包括：

- 替换文本中特殊符号并去除低频词
- 对文本分词
- 构建语料
- 单词映射表

In [3]:
print(words[0:10])
# 筛选低频词  为什么要删除低频词？
words_count = Counter(words)
print('word_count', words_count['java'])
words = [w for w in words if words_count[w] > 50]

['熟练掌握', 'java', '熟悉', 'python', 'shell', '熟练使用', 'git', 'svn', '能够', '发现']
word_count 16940


In [4]:
# 构建映射表
vocab = set(words)
#由于vocab是一个set，因此每次重启kernel后运行代码，w，c的映射关系会不同。而且vocab中不会有重复的单词
vocab_to_int = {w: c for c, w in enumerate(vocab)}  ##返回一个w为key，c为value的字典
int_to_vocab = {c: w for c, w in enumerate(vocab)}

In [5]:
print("total words: {}".format(len(words)))
print("unique words: {}".format(len(set(words))))

total words: 8614953
unique words: 6846


In [6]:
# 对原文本进行vocab到int的转换
int_words = [vocab_to_int[w] for w in words]   #因为是便利words，所以是对原文本
int_java = vocab_to_int['java']
print(int_java)  ##即用2402这个int型数来代表‘java’

374


# 4 采样

对停用词进行采样，例如“the”， “of”以及“for”这类单词进行剔除。剔除这些单词以后能够加快我们的训练过程，同时减少训练过程中的噪音。

我们采用以下公式:
$$ P(w_i) = 1 - \sqrt{\frac{t}{f(w_i)}} $$

其中$ t $是一个阈值参数，一般为1e-3至1e-5。  
$f(w_i)$ 是单词 $w_i$ 在整个数据集中的出现频次。  
$P(w_i)$ 是单词被删除的概率。

>这个公式和论文中描述的那个公式有一些不同

In [7]:
t = 1e-5 # t值
threshold = 0.9 # 剔除概率阈值

# 统计单词出现频次
int_word_counts = Counter(int_words)
print(int_word_counts[2402])   #2402代表’java‘，与words_count['java'])结果都为16940
total_count = len(int_words)  
# 计算单词频率  
word_freqs = {w: c/total_count for w, c in int_word_counts.items()}  ##dict.items用来遍历字典的key和value
# 计算被删除的概率
prob_drop = {w: 1 - np.sqrt(t / word_freqs[w]) for w in int_word_counts}
# 对单词进行采样
train_words = [w for w in int_words if prob_drop[w] < threshold]

156


In [8]:
len(train_words)

3905916

# 5 构造batch

Skip-Gram模型是通过输入词来预测上下文。因此我们要构造我们的训练样本，具体思想请参考知乎专栏，这里不再重复。

对于一个给定词，离它越近的词可能与它越相关，离它越远的词越不相关，这里我们设置窗口大小为5，对于每个训练单词，我们还会在[1:5]之间随机生成一个整数R，用R作为我们最终选择output word的窗口大小。这里之所以多加了一步随机数的窗口重新选择步骤，是为了能够让模型更聚焦于当前input word的邻近词。

In [9]:
def get_targets(words, idx, window_size=5):
    '''
    获得input word的上下文单词列表
    
    参数
    ---
    words: 单词列表(一个batch）
    idx: input word的索引号
    window_size: 窗口大小
    '''
    target_window = np.random.randint(1, window_size+1)
    # 这里要考虑input word前面单词不够的情况
    start_point = idx - target_window if (idx - target_window) > 0 else 0
    end_point = idx + target_window
    # output words(即窗口中的上下文单词) 一个窗口单词个数是 1 + target_window *2
    targets = set(words[start_point: idx] + words[idx+1: end_point+1]) #因为start_point 取值不确定所以要分开写
    return list(targets)

In [10]:
def get_batches(words, batch_size, window_size=5):
    '''
    构造一个获取batch的生成器
    '''
    n_batches = len(words) // batch_size
    
    # 仅取full batches
    words = words[:n_batches*batch_size]  ##超出n_batches*batch_size的部分被舍弃
    print('words[0:1]', words[0:1])
    
    for idx in range(0, len(words), batch_size):
        x, y = [], []
        batch = words[idx: idx+batch_size]   ##一个batch的len始终是batch_size
        for i in range(len(batch)):
            batch_x = batch[i]   ##i的取值范围刚好是0～batch_size
            batch_y = get_targets(batch, i, window_size)
            # 由于一个input word会对应多个output word，因此需要长度统一
            x.extend([batch_x]*len(batch_y))
            y.extend(batch_y)
       # print('x[0:1]=', x[0:1])
       # print('len(x)', len(x))
      #  print('y[0:1]=', y[0:1] )
        yield x, y

# 6 构建网络

该部分主要包括：

- 输入层
- Embedding
- Negative Sampling

## 输入

In [11]:
train_graph = tf.Graph()
with train_graph.as_default():
    inputs = tf.placeholder(tf.int32, shape=[None], name='inputs')
    labels = tf.placeholder(tf.int32, shape=[None, None], name='labels')

## Embedding

嵌入矩阵的矩阵形状为 $ vocab\_size\times hidden\_units\_size$ 

TensorFlow中的tf.nn.embedding_lookup函数可以实现lookup的计算方式

In [12]:
vocab_size = len(int_to_vocab)
print(vocab_size)
embedding_size = 200 # 嵌入维度

6846


In [13]:
with train_graph.as_default():
    # 嵌入层权重矩阵
    embedding = tf.Variable(tf.random_uniform([vocab_size, embedding_size], -1, 1))
    # 实现lookup
    embed = tf.nn.embedding_lookup(embedding, inputs)

## Negative Sampling

负采样主要是为了解决梯度下降计算速度慢的问题，详情同样参考我的上一篇知乎专栏文章。

TensorFlow中的tf.nn.sampled_softmax_loss会在softmax层上进行采样计算损失，计算出的loss要比full softmax loss低。

In [14]:
n_sampled = 100

with train_graph.as_default():
    softmax_w = tf.Variable(tf.truncated_normal([vocab_size, embedding_size], stddev=0.1))
    softmax_b = tf.Variable(tf.zeros(vocab_size))
    
    # 计算negative sampling下的损失
   #loss = tf.nn.sampled_softmax_loss(softmax_w, softmax_b, labels, embed, n_sampled, vocab_size)
    loss = tf.nn.nce_loss(softmax_w, softmax_b, labels, embed, n_sampled, vocab_size)
    cost = tf.reduce_mean(loss)
    optimizer = tf.train.AdamOptimizer().minimize(cost)

## 验证

为了更加直观的看到我们训练的结果，我们将查看训练出的相近语义的词。

In [15]:
with train_graph.as_default():
    # 随机挑选一些单词
    ## From Thushan Ganegedara's implementation
    valid_size = 7 # Random set of words to evaluate similarity on.
    valid_window = 100
    # pick 8 samples from (0,100) and (1000,1100) each ranges. lower id implies more frequent 
    valid_examples = np.array(random.sample(range(valid_window), valid_size//2))
    valid_examples = np.append(valid_examples, 
                               random.sample(range(1000,1000+valid_window), valid_size//2))
    print(valid_examples)
    valid_examples = [vocab_to_int['word'], 
                      vocab_to_int['ppt'], 
                      vocab_to_int['熟悉'],
                      vocab_to_int['java'], 
                      vocab_to_int['能力'], 
                      vocab_to_int['逻辑思维'],
                      vocab_to_int['了解']]
    print(valid_examples)
    valid_size = len(valid_examples)
    # 验证单词集
    valid_dataset = tf.constant(valid_examples, dtype=tf.int32)
    print('valid_dataset', valid_dataset.shape)
    # 计算每个词向量的模并进行单位化
    norm = tf.sqrt(tf.reduce_sum(tf.square(embedding), 1, keep_dims=True))
    normalized_embedding = embedding / norm
    print('normalized_embedding', normalized_embedding.shape)
    # 查找验证单词的词向量
    valid_embedding = tf.nn.embedding_lookup(normalized_embedding, valid_dataset)
    print('valid_embedding', valid_embedding.shape)
    # 计算余弦相似度(验证的6个单词的词向量与整个语料库6846个单词的词向量的相似度)
    similarity = tf.matmul(valid_embedding, tf.transpose(normalized_embedding))
    print(similarity.shape)

[  40   50   84 1039 1027 1025]
[123, 997, 4895, 374, 2357, 5104, 4734]
valid_dataset (7,)
Instructions for updating:
keep_dims is deprecated, use keepdims instead
normalized_embedding (6846, 200)
valid_embedding (7, 200)
(7, 6846)


In [16]:
epochs = 1 # 迭代轮数
batch_size = 1000 # batch大小
window_size = 10 # 窗口大小

with train_graph.as_default():
    saver = tf.train.Saver() # 文件存储

with tf.Session(graph=train_graph) as sess:
    iteration = 1
    loss = 0
    sess.run(tf.global_variables_initializer())

    for e in range(1, epochs+1):
        #带有yield关键字的函数返回一个生成器
        batches = get_batches(train_words, batch_size, window_size)
        print(batches)
        start = time.time()
        # for循环会自动调用next函数
        for x, y in batches:
            
            feed = {inputs: x,
                    labels: np.array(y)[:, None]}
            train_loss, _ = sess.run([cost, optimizer], feed_dict=feed)
            
            loss += train_loss
            
            if iteration % 100 == 0: 
                end = time.time()
                print("Epoch {}/{}".format(e, epochs),
                      "Iteration: {}".format(iteration),
                      "Avg. Training loss: {:.4f}".format(loss/100),
                      "{:.4f} sec/batch".format((end-start)/100))
                loss = 0
                start = time.time()
            
            # 计算相似的词
            if iteration % 1000 == 0:
                # 计算similarity
                sim = similarity.eval()
                for i in range(valid_size):
                    valid_word = int_to_vocab[valid_examples[i]]
                    top_k = 8 # 取最相似单词的前8个 sim前取负号是因为sim越大，相关性越高。则-sim越小，相关新越高
                    nearest = (-sim[i, :]).argsort()[1:top_k+1] #0的位置是valid_word本身
                    log = 'Nearest to [%s]:' % valid_word
                    for k in range(top_k):
                        close_word = int_to_vocab[nearest[k]]
                        log = '%s %s,' % (log, close_word)
                    print(log)
            
            iteration += 1
            
    save_path = saver.save(sess, "checkpoints/text8.ckpt")
    embed_mat = sess.run(normalized_embedding)

<generator object get_batches at 0x7f753d50d7d8>
words[0:1] [2093]
Epoch 1/1 Iteration: 100 Avg. Training loss: 298.9031 0.0835 sec/batch
Epoch 1/1 Iteration: 200 Avg. Training loss: 281.6870 0.0703 sec/batch
Epoch 1/1 Iteration: 300 Avg. Training loss: 236.1611 0.0746 sec/batch
Epoch 1/1 Iteration: 400 Avg. Training loss: 200.3986 0.0695 sec/batch
Epoch 1/1 Iteration: 500 Avg. Training loss: 171.7265 0.0655 sec/batch
Epoch 1/1 Iteration: 600 Avg. Training loss: 143.0911 0.0695 sec/batch
Epoch 1/1 Iteration: 700 Avg. Training loss: 112.2309 0.0752 sec/batch
Epoch 1/1 Iteration: 800 Avg. Training loss: 95.5553 0.0693 sec/batch
Epoch 1/1 Iteration: 900 Avg. Training loss: 88.6673 0.0683 sec/batch
Epoch 1/1 Iteration: 1000 Avg. Training loss: 74.1558 0.0640 sec/batch
[-0.10314064 -0.15883324 -0.03447697]
Nearest to [word]: 乐观, 整理, 团建, python, 待遇, 自信, 扎实, 求知欲,
[-0.28697851 -0.12442343  0.00517113]
Nearest to [ppt]: 内部, 福利待遇, 文字, 经理, 准确, 国内, 完整, 程序,
[-0.04279021 -0.02088061  0.04817206]
Nea